# Trabajo IA

## Selección de características para mejorar modelos predictivos

En este cuadernillo se realiza el tratamiento de datos del csv TITANIC, así como la implementación y comparación de varios algoritmos de búsqueda y entrenamiento.

### Tratamiento de datos

Se aplicaron los siguientes procesos de preprocesamiento de datos:

    •Normalización de variables predictoras: Se normalizaron las variables Age y Fare utilizando el escalador MinMaxScaler para asegurar que todas las características estén en la misma escala.

    •Codificación numérica de atributos discretos: Los atributos Sex, Embarked, Alone y Deck, que originalmente se presentaban como cadenas de texto, fueron codificados numéricamente utilizando las técnicas de OrdinalEncoder o LabelEncoder, según corresponda.

### Primer experimento para evaluar la capacidad predictiva del conjunto de variables completo

    Para evaluar la calidad de las soluciones que se obtienen con los algoritmos de búsqueda de variables, se realizó un primer experimento en el que se entrenó un modelo de clasificación utilizando todas las variables predictoras disponibles. Se utilizó un árbol de decisión de clasificación.

### Algoritmos de búsqueda implementados

    •Búsqueda secuencial hacia atrás (backward_sequential_search): Este algoritmo busca encontrar el mejor subconjunto de variables predictoras eliminando iterativamente la variable que más afecta el rendimiento del modelo.

    •Búsqueda secuencial hacia atrás mixta (backward_sequential_search_mixto): Similar al anterior, pero también considera añadir variables si se mejora el rendimiento del modelo.

### Algoritmos de entrenamiento

    •Árboles de decisión de clasificación (DecisionTreeClassifier): Un algoritmo de aprendizaje supervisado utilizado para clasificación.




## Tratamiento de datos

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler



model = RandomForestClassifier()

titanic = pd.read_csv('../data/titanic.csv')
titanic.head()

atributos_discretos = ['Initial', 'SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married','Pclass', 'Parch', 'Embarked', 
                       'Age_band', 'Family_Size', 'Alone']
atributos_continuos = ['Age', 'Fare']
atributos = titanic.loc[:, atributos_discretos + atributos_continuos]

objetivo = titanic['Survived']
objetivo.head()  # objetivo es una Series unidimensional


codificador_atributos_discretos = OrdinalEncoder()
codificador_atributos_discretos.fit(atributos[atributos_discretos])

print('Número de atributos detectados discretos:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados discretos:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

atributos[atributos_discretos] = codificador_atributos_discretos.transform(atributos[atributos_discretos])
atributos.head()


# El método fit_transform ajusta el codificador a los datos y, a continuación, codifica estos adecuadamente. 

normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)

# Como nos interesa conservar los atributos originales, realizamos la
# normalización sobre una copia del DataFrame de atributos
atributos_normalizados = atributos.copy()
atributos_normalizados[:] = normalizador.fit_transform(atributos_normalizados)
atributos_normalizados.head()

titanic = atributos_normalizados.copy()
titanic['Survived'] = objetivo
titanic.head()


Número de atributos detectados discretos: 13

Nombres de los atributos detectados discretos:
['Initial' 'SibSp' 'Deck' 'Fare_cat' 'Title' 'Sex' 'Is_Married' 'Pclass'
 'Parch' 'Embarked' 'Age_band' 'Family_Size' 'Alone']

Categorías detectadas de cada atributo:
Initial: [0 1 2 3 4]
SibSp: [0 1 2 3 4 5 8]
Deck: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T' 'U']
Fare_cat: [0 1 2 3]
Title: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
Sex: ['female' 'male']
Is_Married: [0 1]
Pclass: [1 2 3]
Parch: [0 1 2 3 4 5 6]
Embarked: ['C' 'Q' 'S']
Age_band: [0 1 2 3 4]
Family_Size: [ 0  1  2  3  4  5  6  7 10]
Alone: ['No' 'Yes']


,Initial,SibSp,Deck,Fare_cat,Title,Sex,Is_Married,Pclass,Parch,Embarked,Age_band,Family_Size,Alone,Age,Fare,Survived
0,0.00,0.166667,1.00,0.000000,0.6875,1.0,0.0,1.0,0.0,1.0,0.25,0.125,0.0,0.271174,0.014151,0
1,0.25,0.166667,0.25,1.000000,0.7500,0.0,1.0,0.0,0.0,0.0,0.50,0.125,0.0,0.472229,0.139136,1
2,0.50,0.000000,1.00,0.333333,0.5000,0.0,0.0,1.0,0.0,1.0,0.25,0.000,1.0,0.321438,0.015469,1
3,0.25,0.166667,0.25,1.000000,0.7500,0.0,1.0,0.0,0.0,1.0,0.50,0.125,0.0,0.434531,0.103644,1
4,0.00,0.000000,1.00,0.333333,0.6875,1.0,0.0,1.0,0.0,1.0,0.50,0.000,1.0,0.434531,0.015713,0


### Evaluación del modelo


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate


(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

clasificador_CART = DecisionTreeClassifier(random_state=42)
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'min_samples_split': range(5, 20, 5)
}
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)
# Obtener los mejores parámetros de la búsqueda en rejilla
mejores_parametros = búsqueda_en_rejilla.best_params_

# Guardar el valor de max_depth en una variable
max_depth_mejor = mejores_parametros.get('max_depth')
min_samples_split_mejor = mejores_parametros.get('min_samples_split')
mejor_score = búsqueda_en_rejilla.best_score_


print("Mejor valor de max_depth:", max_depth_mejor)
print("Mejor valor de min_samples_split:", min_samples_split_mejor)
print("Mejor score:", mejor_score)


Mejor valor de max_depth: 3
Mejor valor de min_samples_split: 5
Mejor score: 0.7915718648276788


In [4]:
resultados_validación_cruzada = cross_validate(clasificador_CART,
                                               atributos_entrenamiento,
                                               objetivo_entrenamiento,
                                               scoring='balanced_accuracy',
                                               cv=10, 
                                               n_jobs=-1,)
#resultados_validación_cruzada
resultados_validación_cruzada['test_score'].mean()

0.7621623208832512

In [10]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score

# Ajustar el modelo
clasificador_CART = DecisionTreeClassifier(max_depth=max_depth_mejor, min_samples_split=min_samples_split_mejor, random_state=42)
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Calcular la Tasa de acierto del modelo
score = clasificador_CART.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_CART.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

# Calcular la sensibilidad
predicciones_prueba = clasificador_CART.predict(atributos_prueba)
recallscore = recall_score(objetivo_prueba, predicciones_prueba)

#Calcular la tasa de acierto balanceada
balanced_accuracy = balanced_accuracy_score(objetivo_prueba, predicciones_prueba)


print(f'Variabes predictoras: \n{atributos_entrenamiento.columns}')
print()
print(f'Tasa de acierto: {score}')
print()
print(f'Valores: {values}')
print()
print(f'Matriz de confusión: \n{m_confusion}')
print()
print(f'Sensibilidad del conjunto de pruebas: {recallscore}')




Variabes predictoras: 
Index(['Initial', 'SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married',
       'Pclass', 'Parch', 'Embarked', 'Age_band', 'Family_Size', 'Alone',
       'Age', 'Fare'],
      dtype='object')

Tasa de acierto: 0.8398876404494382

Valores: Survived
0    439
1    273
Name: count, dtype: int64

Matriz de confusión: 
[[397  42]
 [ 72 201]]

Sensibilidad del conjunto de pruebas: 0.7246376811594203

Tasa de acierto balanceada: 0.7895915678524374


### KNN

In [6]:
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer([('normalizador',
                                   MinMaxScaler(feature_range=(0, 1)),
                                   atributos_discretos + atributos_continuos)])

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

tubería_kNN = Pipeline([('preprocesador', normalizador),
                        ('kNN', KNeighborsClassifier())])
rejilla_de_parámetros = {
    # Número de vecinos impar (tarea de clasificación binaria)
    'kNN__n_neighbors': range(1, 10, 2),
    # Considerar las distancias Manhattan y euclídea
    'kNN__metric': ['manhattan', 'euclidean']
}

búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)

búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Guardar el valor de max_depth en una variable
mejores_parametros = búsqueda_en_rejilla.best_params_

n_neighbors_mejor = mejores_parametros.get('kNN__n_neighbors')
metric_mejor = mejores_parametros.get('kNN__metric')
mejor_score = búsqueda_en_rejilla.best_score_

print("Mejor valor de n_neighbors:", n_neighbors_mejor)
print("Mejor valor de mmetric:", metric_mejor)
print("Mejor score:", mejor_score)


Mejor valor de n_neighbors: 3
Mejor valor de mmetric: manhattan
Mejor score: 0.8133371365929506


In [8]:
clasificador_kNN = KNeighborsClassifier(n_neighbors=n_neighbors_mejor, metric=metric_mejor)

clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)


# Calcular la Tasa de acierto del modelo
score = clasificador_kNN.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_kNN.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

# Calcular la sensibilidad
predicciones_prueba = clasificador_kNN.predict(atributos_prueba)
recallscore = recall_score(objetivo_prueba, predicciones_prueba)



print(f'Variabes predictoras: {atributos_entrenamiento.columns}')
print(f'Tasa de acierto: {score}')
print(f'Valores: {values}')
print(f'Matriz de confusión: \n{m_confusion}')
print()
print(f'Sensibilidad del conjunto de pruebas: {recallscore}')



Variabes predictoras: Index(['Initial', 'SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married',
       'Pclass', 'Parch', 'Embarked', 'Age_band', 'Family_Size', 'Alone',
       'Age', 'Fare'],
      dtype='object')
Tasa de acierto: 0.8693820224719101
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[398  41]
 [ 52 221]]

Sensibilidad del conjunto de pruebas: 0.6956521739130435


## Algoritmo de búsqueda hacia atrás

In [9]:
#Esto parametros nos permite tener el minimo num de variables de predictoras y un mayor score
import funciones.BusquedaSecuencialAtras as bsa
bsatras = bsa.backward_sequential_search(titanic, 'Survived', model, 6, 10)
bsatras


KeyboardInterrupt: 

### Árbol de decisión de clasificación(DecisionTreeClassifier)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(bsatras)-1, 1):
    selected_variables = bsatras.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = titanic.loc[:, selected_variables]
    

    # Dividir los datos
    X = titanic[selected_variables]
    objetivo= titanic['Survived']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    clasificador_CART.fit(X_train, y_train)

    búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

    búsqueda_en_rejilla.fit(X_train, y_train)
   # Obtener los mejores parámetros de la búsqueda en rejilla
    mejores_parametros = búsqueda_en_rejilla.best_params_

    # Guardar el valor de max_depth en una variable
    max_depth_mejor = mejores_parametros.get('max_depth')
    min_samples_split_mejor = mejores_parametros.get('min_samples_split')
    mejor_score = búsqueda_en_rejilla.best_score_


    print("Mejor valor de max_depth:", max_depth_mejor)
    print("Mejor valor de min_samples_split:", min_samples_split_mejor)
    print("Mejor score:", mejor_score)



    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=max_depth_mejor,  # Máxima profundidad del árbol
        min_samples_split=min_samples_split_mejor  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)

    # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=max_depth_mejor)
    clasificador_CART.fit(X_train, y_train)

    # Calcular la Tasa de acierto del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_CART.predict(X_test)
    recallscore = recall_score(objetivo_prueba, predicciones_prueba)

    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Tasa de acierto: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()
    print(f'Sensibilidad del conjunto de pruebas: {recallscore}')
  

Mejor valor de max_depth: 9
Mejor valor de min_samples_split: 15
Mejor score: 0.7794902233855723
Variabes predictoras: 
['SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married', 'Pclass', 'Embarked', 'Age_band', 'Family_Size', 'Alone']
Tasa de acierto: 0.9073033707865169
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[428  11]
 [ 55 218]]

Sensibilidad del conjunto de pruebas: 0.3188405797101449
Mejor valor de max_depth: 5
Mejor valor de min_samples_split: 5
Mejor score: 0.7925458908017048
Variabes predictoras: 
['SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married', 'Pclass', 'Embarked', 'Age_band', 'Family_Size', 'Alone', 'Age']
Tasa de acierto: 0.8455056179775281
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[382  57]
 [ 53 220]]

Sensibilidad del conjunto de pruebas: 0.5072463768115942
Mejor valor de max_depth: 4
Mejor valor de min_samples_split: 15
Mejor score: 0.8033442117744443
Variabes predictora

### KNN

In [ ]:
from sklearn.naive_bayes import CategoricalNB


for k in range(0, len(bsatras)-1, 1):

    selected_variables = bsatras.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = titanic.loc[:, selected_variables]

    X = titanic[selected_variables]
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    clasificador_kNN.fit(X_train, y_train)
    
    búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1) 

    clasificador_kNN.fit(X_train, y_train)

    # Calcular la tasa de acierto del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_kNN.predict(X_train)
    recallscore = recall_score(y_train, predicciones_prueba)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Tasa de acierto: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()
    print(f'Sensibilidad del conjunto de pruebas: {recallscore}')

Variabes predictoras: ['SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married', 'Pclass', 'Embarked', 'Age_band', 'Family_Size', 'Alone']
Tasa de acierto: 0.8539325842696629
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[399  40]
 [ 64 209]]

Sensibilidad del conjunto de pruebas: 0.7655677655677655
Variabes predictoras: ['SibSp', 'Deck', 'Fare_cat', 'Title', 'Sex', 'Is_Married', 'Pclass', 'Embarked', 'Age_band', 'Family_Size', 'Alone', 'Age']
Tasa de acierto: 0.8623595505617978
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[403  36]
 [ 62 211]]

Sensibilidad del conjunto de pruebas: 0.7728937728937729
Variabes predictoras: ['SibSp', 'Fare_cat', 'Title', 'Sex', 'Is_Married', 'Pclass', 'Embarked', 'Age_band', 'Family_Size', 'Alone']
Tasa de acierto: 0.8469101123595506
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[405  34]
 [ 75 198]]

Sensibilidad del conjunto de prueba

## Algoritmo de búsqueda hacia atrás mixta

In [ ]:
import funciones.BusquedaSecuencialAtrasMixta as bsam
busq_atras_mixta = bsam.backward_sequential_mixed_search(titanic, 'Survived', model, 12, 10, 10)
busq_atras_mixta

,variables,size,score
14,"[Initial, Embarked, Is_Married, Title, SibSp, ...",9,0.820649
9,"[SibSp, Deck, Fare_cat, Pclass, Age_band, Init...",9,0.820248
8,"[SibSp, Deck, Fare_cat, Sex, Pclass, Age_band,...",9,0.819778
12,"[Fare_cat, Pclass, Initial, Embarked, Is_Marri...",9,0.819305
11,"[Fare_cat, Pclass, Age_band, Initial, Embarked...",9,0.819123
13,"[Fare_cat, Initial, Embarked, Is_Married, Titl...",9,0.819093
15,"[Initial, Embarked, Is_Married, Title, SibSp, ...",10,0.818819
10,"[Deck, Fare_cat, Pclass, Age_band, Initial, Em...",9,0.818590
7,"[SibSp, Deck, Fare_cat, Title, Sex, Pclass, Ag...",10,0.818501
6,"[SibSp, Deck, Fare_cat, Title, Sex, Pclass, Ag...",10,0.817196


### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(busq_atras_mixta)-1, 1):
    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = titanic.loc[:, selected_variables]
    # Realizar la búsqueda secuencial hacia atrás
    model = DecisionTreeClassifier(max_depth=max_depth_mejor)

    # Dividir los datos
    X = titanic[selected_variables]
    objetivo= titanic['Survived']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

    búsqueda_en_rejilla.fit(X_train, y_train)
   # Obtener los mejores parámetros de la búsqueda en rejilla
    mejores_parametros = búsqueda_en_rejilla.best_params_

    # Guardar el valor de max_depth en una variable
    max_depth_mejor = mejores_parametros.get('max_depth')
    min_samples_split_mejor = mejores_parametros.get('min_samples_split')
    mejor_score = búsqueda_en_rejilla.best_score_

    print("Mejor valor de max_depth:", max_depth_mejor)
    print("Mejor valor de min_samples_split:", min_samples_split_mejor)
    print("Mejor score:", mejor_score)


    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=max_depth_mejor,  # Máxima profundidad del árbol
        min_samples_split=min_samples_split_mejor  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)

    # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=max_depth_mejor)
    clasificador_CART.fit(X_train, y_train)

    # Calcular la Tasa de acierto del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_CART.predict(X_test)
    recallscore = recall_score(y_test, predicciones_prueba)

    # Imprimir los resultados
    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Tasa de acierto: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()  
    print(f'Sensibilidad del conjunto de pruebas: {recallscore}')

Mejor valor de max_depth: 9
Mejor valor de min_samples_split: 15
Mejor score: 0.8053536472141124
Variabes predictoras: 
['Initial', 'Embarked', 'Is_Married', 'Title', 'SibSp', 'Deck', 'Age_band', 'Pclass', 'Fare_cat']
Tasa de acierto: 0.9044943820224719
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[421  18]
 [ 50 223]]

Sensibilidad del conjunto de pruebas: 0.6811594202898551
Mejor valor de max_depth: 9
Mejor valor de min_samples_split: 15
Mejor score: 0.8224183977090954
Variabes predictoras: 
['SibSp', 'Deck', 'Fare_cat', 'Pclass', 'Age_band', 'Initial', 'Embarked', 'Is_Married', 'Title']
Tasa de acierto: 0.9002808988764045
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[416  23]
 [ 48 225]]

Sensibilidad del conjunto de pruebas: 0.5942028985507246
Mejor valor de max_depth: 9
Mejor valor de min_samples_split: 10
Mejor score: 0.7843411971318949
Variabes predictoras: 
['SibSp', 'Deck', 'Fare_cat', 'Sex', 'Pcla

### KNN

In [ ]:
from sklearn.naive_bayes import CategoricalNB


for k in range(0, len(busq_atras_mixta)-1, 1):

    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = titanic.loc[:, selected_variables]

    X = titanic[selected_variables]
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    clasificador_kNN.fit(X_train, y_train)
    
    búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1) 

    clasificador_kNN.fit(X_train, y_train)

    # Calcular la tasa de acierto del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_kNN.predict(X_train)
    recallscore = recall_score(y_train, predicciones_prueba)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Tasa de acierto: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()
    print(f'Sensibilidad del conjunto de pruebas: {recallscore}')

Variabes predictoras: ['Initial', 'Embarked', 'Is_Married', 'Title', 'SibSp', 'Deck', 'Age_band', 'Pclass', 'Fare_cat']
Tasa de acierto: 0.8553370786516854
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[415  24]
 [ 79 194]]

Sensibilidad del conjunto de pruebas: 0.7106227106227107
Variabes predictoras: ['SibSp', 'Deck', 'Fare_cat', 'Pclass', 'Age_band', 'Initial', 'Embarked', 'Is_Married', 'Title']
Tasa de acierto: 0.8707865168539326
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[410  29]
 [ 63 210]]

Sensibilidad del conjunto de pruebas: 0.7692307692307693
Variabes predictoras: ['SibSp', 'Deck', 'Fare_cat', 'Sex', 'Pclass', 'Age_band', 'Initial', 'Embarked', 'Is_Married']
Tasa de acierto: 0.8665730337078652
Valores: Survived
0    439
1    273
Name: count, dtype: int64
Matriz de confusión: 
[[400  39]
 [ 56 217]]

Sensibilidad del conjunto de pruebas: 0.7948717948717948
Variabes predictoras: ['Fare_cat', 'Pcl